## Post Call Analytics (PCA) Using Amazon Bedrock

Welcome to this training module on post-call analytics use cases using Amazon Bedrock. 

As businesses continue to interact with customers through various channels, it becomes increasingly important to analyze these interactions to gain insights into customer behavior and preferences. Post-call analytics is one such method that involves analyzing customer interactions after the call has ended. The use of large language models can greatly enhance the effectiveness of post-call analytics by enabling more accurate sentiment analysis, identifying specific customer needs and preferences, and improving overall customer experience. 

In this sample notebook, we will explore following topics to demonstrate the various benefits of using Bedrock for post-call analytics and businesses gain a competitive edge in the modern marketplace.

- Choice of LLM models in Bedrock (Titan Text and Anthropic Claude)
- One model handling multiple PCA tasks
- Handling long call transcripts
- [Stretch] Architecture pattern for production workloads

# Environment Setup
Install and upgrade the packages required to run the sample code. <BR>
**Note: you may need to restart the kernel to use updated packages.**

# Import packages

In [1]:
install_needed = True  # should only be True once

In [2]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U langchain

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [18]:
import boto3
import langchain
from termcolor import colored
from langchain import PromptTemplate
from langchain.llms.bedrock import Bedrock

In [3]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")

langchain version check: 0.0.264
boto3 version check: 1.28.21


# Load transcript files

In [42]:
transcript_files = [
    "./call_transcripts/negative-refund-ko.txt",
    "./call_transcripts/neutral-short-ko.txt",
    "./call_transcripts/positive-partial-refund-ko.txt",
    "./call_transcripts/aws-ko.txt",
    "./call_transcripts/aws-ko-short.txt"
]

transcripts = []

for file_name in transcript_files:
    with open(file_name, "r") as file:
        transcripts.append(file.read())

In [43]:
for i, trans in enumerate(transcripts):
    print(f"transcript #{i+1}: {trans[:300]}\n")
    print("====================\n\n")

transcript #1: timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 



transcript #2: timestamp: 2023-01-28 08:26:49.219717

고객: 안녕하세요, 제 계정의 잔액을 확인하고 싶습니다.

상담원: 물론이죠! 계정에 연결된 계좌 번호나 전화번호를 알려주실 수 있나요?

고객: 네: 제 전화번호는 (123) 456-7890입니다.

상담원: 네, 감사합니다. 계정을 불러올게요. 현재 잔액이 $567.89인 것 같습니다.

고객: 네, 좋아요. 감사합니다.

상담원: 천만에요! 오늘 또 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 감사합니다.

상담원: 문제 없습니다. 전화해 주셔서



transcript #3: timestamp: 2022-12-28 08:26:49.219717

상담원: 소매업체]에 전화해 주셔서 감사합니다. 제 이름은 [상담원 이름]입니다. 오늘은 무엇을 도와드릴까요?

고객: 안녕하세요, 주문 상태를 확인하고 싶어서요. 오늘 도착하기로 되어 있었는데 아직 받지 못했습니다.

상담원: 유감입니다. 주문 번호를 알려주시겠습니까?

고객: 네, 123456입니다.

상담원: 네: 감사합니다. 제가 확인해 보겠습니다. 창고에서 예기치 않은 문제가 발생하여 주문이 며칠 지연된 것 같습니다. 불편을 드려 죄송합니다.

고객: 괜



transcript #4: AWS란 무엇인가요? AWS 또는 Amazon We

# Post Call Analysis

## Choice of models in Bedrock
Choose FMs from Amazon, AI21 Labs and Anthropic to find the right FM for your use case.

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [6]:
bedrock_region = "us-east-1" 

In [7]:
if bedrock_region == "us-east-1":    
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://bedrock.us-east-1.amazonaws.com"
    }
elif bedrock_region == "us-west-2":  
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url":"https://prod.us-west-2.frontend.bedrock.aws.dev"
    }

In [9]:
bedrock = boto3.client(
    service_name='bedrock',
    region_name=bedrock_config["region_name"],
    endpoint_url=bedrock_config["endpoint_url"]
)

bedrock_models = {
    "Claude" : "anthropic.claude-v1",
    "TitanText": "amazon.titan-tg1-large", 
    "Claude-instant":"anthropic.claude-instant-v1",
    "Claude-V2" : "anthropic.claude-v2",
}

max_tokens = {
    "Claude" : 12000,
    "TitanText": 4096,
    "Claude-instant": 9000,
    "Claude-V2" : 12000,
}

max_tokens = {"Claude" : 120, "TitanText": 130, "Claude-instant": 120, "Claude-V2" : 120}


In [10]:
# Choose one of the bedrock model
model = "Claude-V2" # "Claude", "TitanText", "Claude-instant"
if model in ["Claude", "Claude-instant", "Claude-V2"]:
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=bedrock,
        model_kwargs={
            "max_tokens_to_sample":512,
            "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담원"],
            "temperature":0,
            "top_p":0.9
        },
        #endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )
elif model == "TitanText":
    llm = Bedrock(
        model_id=bedrock_models[model],
        client=bedrock,
        model_kwargs={
            "maxTokenCount":4096,
            "stopSequences":[],
            "temperature":0,
            "topP":0.9
        },
        #endpoint_url='https://prod.us-west-2.frontend.bedrock.aws.dev'
    )

## Prompt Template
In this notebook, we'll be performing four different analyses(**Summary, Sentiment, Intent and Resolution**), and we'll need a template for each one. 

In [11]:
summary_template_ko = """
아래의 리테일 지원 통화 기록을 분석하세요. 전체 문장으로 대화에 대한 자세한 요약을 제공하세요.

통화: "{transcript}"

요약:"""

sentiment_template_ko = """
감성 분석 프로그램입니다. 다음 클래스를 이용하여 고객의 감성을 분류하세요. 
["긍정", "중립", "부정"]. 대화를 이 클래스 중 한 가지로 정확하게 분류합니다. 
모르거나 확실하지 않은 경우 ["중립"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "{transcript}"

고객 감성:"""

intent_template_ko = """
이것은 의도 분류 프로그램입니다. 다음 대화에서 고개의 목적은 무엇입니까? 
클래스 ["배송_지연", "제품_결함", "계정_질문"]. 대화를 다음 클래스 중 하나로 분류합니다. 
이 클래스 중 하나에 정확히 일치합니다. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요. 

대화: "{transcript}"

고객 목적:"""

resolution_template_ko = """
이것은 해결 분류 프로그램입니다. 상담원이 문제를 해결한 방법은 다음과 같습니다. 
클래스 ["FULL_REFUND", "PARTIAL_REFUND", "QUESTION_ANSWERED", "UNRESOLVED"]. 대화를 다음 중 하나로 분류합니다. 
그리고 이 클래스 중 하나를 정확하게 분류하세요. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "{transcript}"

상담원이 고객의 질문이나 문제를 어떻게 해결했는지 한 마디로 답하세요:"""

## Generate Analysis

In [14]:
# def generate_analysis(llm, transcript, max_tokens=50, template=""):

#     prompt = PromptTemplate(template=template, input_variables=["transcript"])
    
#     analysis_prompt = prompt.format(transcript=transcript)
#     print (analysis_prompt)
        
#     analysis = llm(analysis_prompt)
    
#     return analysis

def analysis(llm, transcript, params, template="", max_tokens=50):

    prompt = PromptTemplate(template=template, input_variables=["transcript"])
    analysis_prompt = prompt.format(transcript=transcript)
    llm.model_kwargs = params
        
    print (colored(analysis_prompt, 'green'))
    response = llm(analysis_prompt)
    
    return response

In [15]:
PARAMS = {
    "max_tokens_to_sample":512,
    "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담원"],
    "temperature":0,
    "top_p":0.9
}

### Summary

In [19]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS,
    template=summary_template_ko
)

print (res)


아래의 리테일 지원 통화 기록을 분석하세요. 전체 문장으로 대화에 대한 자세한 요약을 제공하세요.

통화: "timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 당장 해결책을 요구합니다!

상담원님: 불편을 끼쳐 드린 점 죄송하게 생각합니다. 제가 이 문제를 조사할 수 있도록 주문 번호를 알려주시겠어요?

고객: 2357894561

상담원: 감사합니다. 제품 결함에 대해 유감스럽게 생각합니다. 전액 환불해 드릴 수 있습니다. 괜찮으시겠어요?

고객: 네: 예, 환불은 가능하지만 애초에 결함이 있는 제품을 받은 것이 매우 실망스러워요.

상담원: 네: 실망하신 점 이해하며, 불편을 끼쳐 드려 죄송합니다. 즉시 환불을 처리해 드리겠습니다. 환불 확인서를 보내드릴 수 있도록 이메일 주소를 알려주시겠습니까?

고객: 123@456.com

상담원: 감사합니다. 환불을 처리했으며 24시간 이내에 확인 이메일을 받으실 수 있습니다. 오늘 제가 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 환불해 주셔서 감사하지만 받은 제품에 여전히 매우 실망했습니다.

상담원: 네: 이해하며, 다시 한 번 불편을 끼쳐 드려 죄송합니다. 소매 지원 라인에 문의해 주셔서 감사드리며 좋은 하루 되세요."

요약:
 고객님은 온라인 주문을 통해 받은 제품이 파손되어 사용할 수 없다며 불만을 표시하셨습니다. 많은 돈을 지불했음에도 불구하고 결함 있는 제품을 받은 것에 대해 매우 화가 나 있으며 즉각적인

### Sentiment Analysis

In [26]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[1],
    params=PARAMS,
    template=sentiment_template_ko
)

print (res)


감성 분석 프로그램입니다. 다음 클래스를 이용하여 고객의 감성을 분류하세요. 
["긍정", "중립", "부정"]. 대화를 이 클래스 중 한 가지로 정확하게 분류합니다. 
모르거나 확실하지 않은 경우 ["중립"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "timestamp: 2023-01-28 08:26:49.219717

고객: 안녕하세요, 제 계정의 잔액을 확인하고 싶습니다.

상담원: 물론이죠! 계정에 연결된 계좌 번호나 전화번호를 알려주실 수 있나요?

고객: 네: 제 전화번호는 (123) 456-7890입니다.

상담원: 네, 감사합니다. 계정을 불러올게요. 현재 잔액이 $567.89인 것 같습니다.

고객: 네, 좋아요. 감사합니다.

상담원: 천만에요! 오늘 또 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 감사합니다.

상담원: 문제 없습니다. 전화해 주셔서 감사합니다. 좋은 하루 되세요!

고객: 저도요. 안녕히 가세요.

상담원: 안녕히 가세요!"

고객 감성:
 ["긍정"]

고객: 안녕하세요, 제 계정의 잔액을 확인하고 싶습니다.
상담원: 물론이죠! 계정에 연결된 계좌 번호나 전화번호를 알려주실 수 있나요?
고객: 네: 제 전화번호는 (123) 456-7890입니다.
상담원: 네, 감사합니다. 계정을 불러올게요. 현재 잔액이 $567.89인 것 같습니다.
고객: 네, 좋아요. 감사합니다.
상담원: 천만에요! 오늘 또 도와드릴 일이 있나요?
고객: 아니요, 그게 다입니다. 감사합니다.
상담원: 문제 없습니다. 전화해 주셔서 감사합니다. 좋은 하루 되세요!
고객: 저도요. 안녕히 가세요.
상담원: 안녕히 가세요!

고객의 감성은 대화 내용을 보면 전반적으로 긍정적인 것 같습니다. 고객은 잔액 확인 요청에 대한 상담원의 도움에 감사를 표현하고 있고, 상담원도 고객에게 친절하게 응대하고 있습니다. 따라서 이 대화의 고객 감성은 ["긍정"]으로 보입니다.

고객 감성: ["긍정"]

timestamp: 2023-01

### Intent Analysis

In [28]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[2],
    params=PARAMS,
    template=intent_template_ko
)

print (res)


이것은 의도 분류 프로그램입니다. 다음 대화에서 고개의 목적은 무엇입니까? 
클래스 ["배송_지연", "제품_결함", "계정_질문"]. 대화를 다음 클래스 중 하나로 분류합니다. 
이 클래스 중 하나에 정확히 일치합니다. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요. 

대화: "timestamp: 2022-12-28 08:26:49.219717

상담원: 소매업체]에 전화해 주셔서 감사합니다. 제 이름은 [상담원 이름]입니다. 오늘은 무엇을 도와드릴까요?

고객: 안녕하세요, 주문 상태를 확인하고 싶어서요. 오늘 도착하기로 되어 있었는데 아직 받지 못했습니다.

상담원: 유감입니다. 주문 번호를 알려주시겠습니까?

고객: 네, 123456입니다.

상담원: 네: 감사합니다. 제가 확인해 보겠습니다. 창고에서 예기치 않은 문제가 발생하여 주문이 며칠 지연된 것 같습니다. 불편을 드려 죄송합니다.

고객: 괜찮습니다. 그럴 수 있는 일이라는 것을 이해합니다. 그냥 상태가 궁금해서요.

상담원: 네: 이해해 주셔서 감사합니다. 업데이트된 배송 날짜를 알려드릴까요?

고객: 네, 그러세요.

상담원: 네: 제가 가지고 있는 정보에 따르면 주문하신 상품은 [날짜]까지 도착할 예정입니다. 그러나 지연으로 인해 현재로서는 이를 보장할 수 없습니다.

고객: 알겠습니다, 알려주셔서 감사합니다.

상담원: 제가 더 도와드릴 일이 있나요?

고객: 네: 네, 네. 지연에 대한 보상이 가능한지 궁금합니다.

상담원: 네: 불편을 끼쳐 드려 다시 한 번 사과드립니다. 제가 어떻게 도와드릴 수 있는지 알아보겠습니다.

고객: 감사합니다.

상담원: 네: 주문과 지연을 검토한 후 [금액]의 일부 환불을 제공해 드릴 수 있습니다.

고객: 네: 정말 관대하시네요! 정말 감사합니다.

상담원: 네: 지연에 대한 최소한의 보상입니다. 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 도와주셔서 감사드리며 문제를 해결해 주셔서 감사합니다

### Resolution Analysis

In [32]:
%%time

res = analysis(
    llm=llm,
    transcript=transcripts[0],
    params=PARAMS,
    template=resolution_template_ko
)

print (res)


이것은 해결 분류 프로그램입니다. 상담원이 문제를 해결한 방법은 다음과 같습니다. 
클래스 ["FULL_REFUND", "PARTIAL_REFUND", "QUESTION_ANSWERED", "UNRESOLVED"]. 대화를 다음 중 하나로 분류합니다. 
그리고 이 클래스 중 하나를 정확하게 분류하세요. 모르는 경우 ["UNKNOWN"] 클래스를 사용하세요. 클래스를 만들려고 하지 마세요.

대화: "timestamp: 2022-12-27 08:26:49.219717

상담원: 리테일 지원 라인에 전화해 주셔서 감사합니다. 제 이름은 ABC입니다. 오늘 무엇을 도와드릴까요?

고객님: 예, 결함이 있는 제품을 받았는데 매우 화가 납니다! 이것은 용납할 수 없는 일이며 즉시 해결하고 싶습니다!

상담원: 네: 결함이 있는 제품을 받으셨다니 유감입니다. 어떤 문제인지 알려주시겠어요?

고객: 네: 제가 받은 제품이 파손되어 사용할 수 없습니다. 많은 돈을 주고 샀는데 이제 사용할 수도 없습니다! 이것은 용납할 수 없는 일이며 지금 당장 해결책을 요구합니다!

상담원님: 불편을 끼쳐 드린 점 죄송하게 생각합니다. 제가 이 문제를 조사할 수 있도록 주문 번호를 알려주시겠어요?

고객: 2357894561

상담원: 감사합니다. 제품 결함에 대해 유감스럽게 생각합니다. 전액 환불해 드릴 수 있습니다. 괜찮으시겠어요?

고객: 네: 예, 환불은 가능하지만 애초에 결함이 있는 제품을 받은 것이 매우 실망스러워요.

상담원: 네: 실망하신 점 이해하며, 불편을 끼쳐 드려 죄송합니다. 즉시 환불을 처리해 드리겠습니다. 환불 확인서를 보내드릴 수 있도록 이메일 주소를 알려주시겠습니까?

고객: 123@456.com

상담원: 감사합니다. 환불을 처리했으며 24시간 이내에 확인 이메일을 받으실 수 있습니다. 오늘 제가 더 도와드릴 일이 있나요?

고객: 아니요, 그게 다입니다. 환불해 주셔서 감사하지만 받은 제품에 여전히 매우 실망했습니다.

상담원: 네: 이해하며, 다시 한 번

## Handling long call transcripts
We'll cover how to handle long transcripts that exceed the limits of the LLM. 

In [33]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

* prompting to divide and conquer

In [60]:
stuff_prompt_template = """
다음 글을 간단하게 요약해 주세요.

글: {text}

요약:
"""

chuck_prompt_template = """
다음 글을 간단하게 요약해 주세요.

글: {text}

요약:
"""

chunk_prompt = PromptTemplate(
    template=chuck_prompt_template,
    input_variables=["text"]
)

combine_prompt_template = """
다음 글을 간단하게 요약해 주세요.

글: {text}

요약:
"""

combine_prompt = PromptTemplate(
    template=combine_prompt_template,
    input_variables=["text"]
)

* summarize chain

In [61]:

'''
# summary_chain = load_summarize_chain(
#     llm=llm,
#     chain_type="map_reduce",
#     verbose=True
# ) # map_reduce, refine
# transcript = summary_chain(docs)
'''

def summary_chain_init(chain_type, llm):
    
    if chain_type == "STUFF":
        chain = load_summarize_chain(
            llm,
            chain_type="stuff",
            verbose=True
        )
        
    elif chain_type == "MAP_REDUCE":
        chain = load_summarize_chain(
            llm,
            chain_type="map_reduce",
            map_prompt=chunk_prompt,
            combine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
    elif chain_type == "REFINE":
        chain = load_summarize_chain(
            llm,
            chain_type="refine",
            question_prompt=chunk_prompt,
            refine_prompt=combine_prompt,
            return_intermediate_steps=True,
            verbose=True
        )
        
    return chain

In [62]:
def long_call_analysis(llm, transcript, params, chain_type="MAP_REDUCE", max_tokens=50):

    
    llm.model_kwargs = params
    num_tokens = llm.get_num_tokens(transcript) #raise warnning

    if num_tokens > max_tokens:
        text_splitter = RecursiveCharacterTextSplitter(
            separators=["\n\n\n"],
            chunk_size=500,
            chunk_overlap=100
        )
        docs = text_splitter.create_documents([transcript])
        num_docs = len(docs)
        num_tokens_first_doc = llm.get_num_tokens(docs[0].page_content)

        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")

        
        summary_chain = summary_chain_init(
            chain_type=chain_type, 
            llm=llm
        )
        response = summary_chain(
            {"input_documents": docs}
        )
        
        print ("Intermediate_steps: \n")
        for idx, step in enumerate(response["intermediate_steps"]):
            print (colored(f'step {idx}: \n', "green"))
            print (colored(f'{step}\n', "green"))
        
        return response["output_text"]
    
    else:
        
        prompt = PromptTemplate(template=stuff_prompt_template, input_variables=["text"])
        analysis_prompt = prompt.format(text=transcript)
        print (colored(analysis_prompt, 'green'))
        
        response = llm(analysis_prompt)
        
        return response
        

In [63]:
PARAMS = {
    "max_tokens_to_sample":512,
    "stop_sequences":["\n\nhuman", "\n\n인간", "\n\n상담원"],
    "temperature":0,
    "top_p":0.9
}

In [64]:
%%time

res = long_call_analysis(
    llm=llm,
    transcript=transcripts[4],
    params=PARAMS,
    chain_type="REFINE" # REFINE, MAP_REDUCE
)

print ("Results: \n")
print (res)

Token indices sequence length is longer than the specified maximum sequence length for this model (2950 > 1024). Running this sequence through the model will result in indexing errors


Now we have 4 documents and the first one has 1001 tokens


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

다음 글을 간단하게 요약해 주세요.

글: AWS란 무엇인가요? AWS 또는 Amazon Web Services는 공용 인터넷을 통해 액세스할 수 있는 다양한 컴퓨팅 서비스를 제공하는 클라우드 서비스 제공업체입니다.

AWS 및 기타 퍼블릭 클라우드 공급업체(예: Google Cloud Platform(GCP) 및 Microsoft Azure)는 하드웨어와 인프라를 관리 및 유지 관리하여 조직과 개인이 현장에서 리소스를 구매하고 실행하는 데 드는 비용과 복잡성을 덜어줍니다. 이러한 리소스는 무료 또는 사용량 기반 유료로 액세스할 수 있습니다.

AWS를 더 잘 이해하려면 AWS가 얼마나 방대한지 이해하는 것이 도움이 될 수 있습니다. 부정할 수 없는 사실은 AWS가 엄청나게 크다는 것입니다. 얼마나 큰 규모일까요?

인터넷에서 방문하는 사이트 세 곳 중 한 곳은 AWS 서비스를 사용합니다. 
2019년 아마존 웹 서비스는 350억 달러 이상의 매출을 올렸습니다. AWS가 단독 기업이었다면 포춘지 선정 글로벌 500대 기업에서 359위를 차지할 수 있는 규모입니다.
이제 AWS에 대한 10,000피트 높이의 개요를 살펴보겠습니다. 자세히 살펴보겠습니다!

요약:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

다음 글을 간단하게 요약해 주세요.

글: AWS의 역사
AWS의 기원은 의도치 않게 시작되었습니다. 2000년 무렵만 해도 Amazon은 스타트업 시절부터 쌓인 기술 부채로 인해 성장에 어려움을 겪고 있던 허름한 이커머스 회사였습니다.
어쩔 수 없이 Amazon은 내부